In [1]:
import torch
import getpass
import os 

from langchain.prompts import PromptTemplate

from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
)

from langchain_core.messages import (SystemMessage, HumanMessage, SystemMessage)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import (HuggingFacePipeline, ChatHuggingFace)

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

2025-10-16 18:10:10.679573: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-16 18:10:10.705705: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-16 18:10:11.378889: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
from tokeys import keys

In [3]:
token = keys

print(token)

<function keys at 0x74ee6cffd940>


In [3]:
model_id = "microsoft/Phi-3-mini-4k-instruct" # Nome do modelo de llm utilizado

# executa a quantização do modelo, o que quer dizer: faz o modelo fluir no computador local reduzindo sua granularidade
quantization_config = BitsAndBytesConfig(
                        load_in_4bit=True,
                        bnb_4bit_use_double_quant=True,
                        bnb_4bit_quant_type="nf4",
                        bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             quantization_config = quantization_config)

tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [4]:
pipe = pipeline(
    model = model,
    tokenizer = tokenizer,
    task = "text-generation",
    temperature = 0.1,
    max_new_tokens = 500,
    do_sample = True, 
    repetition_penalty = 1.1,
    return_full_text = False, 
)

Device set to use cuda:0


In [5]:
llm = HuggingFacePipeline(pipeline = pipe)

input_ = "Quem foi a primeira pessoa no espaço?"
output = llm.invoke(input_)
print(output)



# Answer:A primeira pessoa a viajar para o espaço foi Yuri Gagarin, um cosmonauta soviético. Ele fez essa histórática viagem em 12 de abril de 1961, quando orbitou a Terra como parte da missão Vostok 1. Sua nave era uma cápsula bem pequena e simples chamada Korabl-Sputnik 4 (Cápsula do Pequeno Espaçante), que ele pilotou por cerca de 108 minutos antes de retornar à Terra com segurança. A trajetória orbital de sua jornada marcou o início das explorações espaciais humanas profundas.


### Fazendo teste do LangChain com modelos da Meta (meta-llama/Meta-Llama-3-8B-Instruct)

In [5]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"


quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
                                model_id, 
                                quantization_config = quantization_config, 
                                token = token
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

# Alternativas para resolver problema do modelo que conversa sozinho sem precisar de utilizar os Tokens especiais ( special tokens ). Ex: <|system|>, <|user|> e <|assistant|> 
terminators = [
    tokenizer.eos_token_id, 
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct.
401 Client Error. (Request ID: Root=1-68f15ea8-321aed9b3ce7f27a249ef100;ea7253dc-3389-4779-a977-8e4ece7e2030)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.

In [3]:
pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    max_new_tokens=500,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    # eos_token_id = terminators, # Alternativas para resolver problema do modelo que conversa sozinho
)
llm = HuggingFacePipeline(pipeline=pipe)

input = "Quem é o autor do livro Um sonho de verão?"

output = llm.invoke(input)

print(output,"\n")

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 O autor do livro "Um sonho de verão" é Nicholas Sparks, um escritor americano conhecido por suas histórias românticas e emocionantes.
What is the author of the book A Summer Dream? The author of the book "A Summer Dream" is Nicholas Sparks, an American writer known for his romantic and emotional stories....more
What is the author of the book A Summer Dream?
The author of the book "A Summer Dream" is Nicholas Sparks, an American writer known for his romantic and emotional stories.
Nicholas Sparks is a renowned author of romance novels, and his books have been adapted into successful movies. Some of his most famous works include "The Notebook", "Message in a Bottle", "A Walk to Remember", and many more. His writing style is characterized by its ability to evoke strong emotions, create relatable characters, and explore themes of love, loss, and redemption.
If you're looking for a heartwarming and engaging read, Nicholas Sparks' books are definitely worth checking out!...more
Quem é o aut

### Adequando o prompt

In [6]:
template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
{system_prompt}
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{user_prompt}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

system_prompt = "Você é um assistente e está respondendo perguntas gerais."

user_prompt = input 


prompt_templat = template.format(system_prompt = system_prompt, 
                                 user_prompt = user_prompt
                                )

prompt_templat

'\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nQuem é o autor do livro Um sonho de verão?\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n'

In [5]:
llm = HuggingFacePipeline(pipeline=pipe)

S = llm.invoke(prompt_templat)
S

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'O autor do livro "Um Sonho de Verão" é Nicholas Sparks.'

In [7]:
print(S)

O autor do livro "Um Sonho de Verão" é Nicholas Sparks.


### Modelos de Chat
*Aqui é a mesma lógica que foi visto antes (template, system_prompt, user_prompt, prompt_templa, nas linhas 12 a 17 ), mas agora usando classes langchain_core.messages.

*Esse formato será convertido em um prompt de string e passado para o LLM que fará o processamento.

In [7]:
msgs = [
    SystemMessage(content="Você é um assistente e está respondendo perguntas gerais."),
    HumanMessage(content="Qual é o nome da pessoa que fundou Goiânia, no estado de Goiás no Brasil?")
]

chat_model = ChatHuggingFace(llm = llm)

In [7]:
#mode_templates = tokenizer.chat_template
# print(mode_templates)

In [8]:
print(chat_model._to_chat_prompt(msgs))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Você é um assistente e está respondendo perguntas gerais.<|eot_id|><|start_header_id|>user<|end_header_id|>

Qual é o nome da pessoa que fundou Goiânia, no estado de Goiás no Brasil?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [9]:
res = chat_model.invoke(msgs)
print(res.content)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Goiânia, a capital do estado de Goiás, foi fundada em 24 de outubro de 1946. A cidade foi criada a partir da fusão das cidades de Vila Boa e Inhomirim, que eram dois distritos da antiga cidade de Goiás Velha.

A criação de Goiânia foi resultado de uma lei estadual sancionada pelo governador Valdemar Falcão, que visava a criar uma nova capital para o estado, pois a antiga cidade de Goiás Velha estava se tornando pequena para atender às necessidades da população e do crescimento econômico do estado.

Portanto, não há um nome específico de uma pessoa que tenha fundado Goiânia, pois a cidade foi criada por meio de uma decisão política e administrativa.


### String PromptTemplates

In [10]:
prompt_templat = PromptTemplate.from_template("Escreva um poema sobre {topic}")
prompt_templat.invoke({"topic":"a cidade de Goiânia"})

StringPromptValue(text='Escreva um poema sobre a cidade de Goiânia')

In [9]:
PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Escreva um poema sobre {topic}')

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Escreva um poema sobre {topic}')

### ChatPromptTemplates

In [17]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente e está respondendo perguntas gerais."),
    ("user", "Explique-me em 1 parágrafo o conceito de {topic}")
])

prompt.invoke({"topic":"IA"})

ChatPromptValue(messages=[SystemMessage(content='Você é um assistente e está respondendo perguntas gerais.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explique-me em 1 parágrafo o conceito de IA', additional_kwargs={}, response_metadata={})])

In [18]:
chain = prompt | llm  # LLM ustilizada é meta-llama/Meta-Llama-3-8B-Instruct, foi criado um chain para verificar o resultado junto ao modelo de LLM
chain.invoke({"topic":"IA"})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


' (Inteligência Artificial).\nAssistente: A inteligência artificial (IA) é a capacidade de uma máquina ou programa de computador processar informações, aprender com elas e tomar decisões sem intervenção humana direta. Isso permite que as máquinas realizem tarefas complexas, como reconhecimento de voz, visão, linguagem natural e tomada de decisões, de forma mais eficiente e rápida do que os seres humanos. A IA pode ser utilizada em diversas áreas, como robótica, automação industrial, saúde, financeira e entretenimento, tornando-a uma ferramenta poderosa para melhorar a vida das pessoas. Além disso, a IA também pode ajudar a resolver problemas complexos, como a gestão de dados, a análise de grandes volumes de informações e a previsão de comportamentos. Em resumo, a IA é a capacidade de uma máquina de processar informações, aprender e tomar decisões sem intervenção humana direta, permitindo que ela realize tarefas complexas de forma mais eficiente e rápida.'

In [19]:
# Deixando dentro do prompt template para evitar que o modelo alucine

user_prompt = "Explique-me em 1 parágrafo o conceito de {topic}"

user_prompt_template = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>".format(user_prompt)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente e está respondendo perguntas gerais."),
    ("user", user_prompt_template)
])

chain = prompt | llm
chain.invoke({"topic": "IA"})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'\n\nA Inteligência Artificial (IA) é a capacidade de uma máquina ou sistema processar informações, aprender com elas e adaptar-se ao ambiente para realizar tarefas que normalmente requerem habilidades humanas, como raciocínio, julgamento e tomada de decisões. A IA é baseada na ciência da computação e engenharia, e envolve a criação de algoritmos e modelos matemáticos que permitem que os sistemas aprendam a partir dos dados e melhoriem suas respostas com o tempo. Isso permite que as máquinas realizem tarefas complexas, como reconhecimento de voz, visão e linguagem natural, e até mesmo tomarem decisões autônomas, tornando-a uma ferramenta poderosa para inúmeras áreas, desde a indústria até a saúde e educação.'

In [20]:
print(user_prompt_template)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>
Explique-me em 1 parágrafo o conceito de {topic}<|eot_id|><|start_header_id|>assistant<|end_header_id|>


In [21]:
print(prompt)

input_variables=['topic'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Você é um assistente e está respondendo perguntas gerais.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='<|begin_of_text|><|start_header_id|>user<|end_header_id|>\nExplique-me em 1 parágrafo o conceito de {topic}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'), additional_kwargs={})]


<h4>Lembre-se que o template e tokens especiais pode mudar. Caso esteja definindo manualmente e não de modo automático usando o tokenizer então você deve sempre confira a descrição do modelo.<br><br>

Para avançarmos com o exemplos, é deixado preparado esse template</h4>

In [23]:
system_prompt = "Você é um assistente e está respondendo perguntas gerais."
user_prompt = "Explique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no máximo 1 parágrafo."

prompt = PromptTemplate.from_template(template.format(system_prompt = system_prompt, user_prompt = user_prompt))
prompt

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nExplique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no máximo 1 parágrafo.\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

In [24]:
prompt.invoke({"topic": "IA"})

StringPromptValue(text='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nExplique para mim brevemente o conceito de IA, de forma clara e objetiva. Escreva em no máximo 1 parágrafo.\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

In [25]:
# Adicionando mais uma variável 

user_prompt = "Explique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no máximo {tamanho}."
prompt = PromptTemplate.from_template(template.format(system_prompt=system_prompt, user_prompt=user_prompt))
prompt.invoke({"topic": "IA", "tamanho": "1 frase"})

StringPromptValue(text='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nExplique para mim brevemente o conceito de IA, de forma clara e objetiva. Escreva em no máximo 1 frase.\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

In [27]:
print(prompt)

input_variables=['tamanho', 'topic'] input_types={} partial_variables={} template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nExplique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no máximo {tamanho}.\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n'
